In [2]:
import scipy.io as sio
import numpy as np
import tensorflow as tf

In [3]:
# Define random seed
np.random.seed(42)
tf.random.set_seed(42)

### Import data

In [4]:
data = sio.loadmat('cylinder_nektar_wake.mat')

In [5]:
U_star = data['U_star'] # N x 2 x T
p_star = data['p_star'] # N x 2 x T
t_star = data ['t'] # T x 1
X_star = data['X_star'] # N x 2 

N = X_star.shape[0] # 5000
T = t_star.shape[0] # 200

### Select test data for the PINN

In [6]:
x_test = X_star[:, 0:1]
y_test = X_star[:, 1:2]
p_test = p_star[:, 0:1]
u_test = U_star[:, 0:1, 0]
t_test = np.ones((x_test.shape[0], x_test.shape[1]))

### Rearrange Data

In [7]:
XX = np.tile(X_star[:, 0:1], (1, T))  # N x T
YY = np.tile(X_star[:, 1:2], (1, T))  # N x T
TT = np.tile(t_star, (1, N)).T  # N x T

UU = U_star[:, 0, :]  # N x T
VV = U_star[:, 1, :]  # N x T
PP = p_star  # N x T

x = XX.flatten()[:, None]  # NT x 1
y = YY.flatten()[:, None]  # NT x 1
t = TT.flatten()[:, None]  # NT x 1

u = UU.flatten()[:, None]  # NT x 1
v = VV.flatten()[:, None]  # NT x 1
p = PP.flatten()[:, None]  # NT x 1

### Select training data for the PINN

In [19]:
# Noiseless data
N_train = 5000
idx = np.random.choice(N*T, N_train, replace=False)
x_train = x[idx,:] # 5000 x 1
y_train = y[idx,:]
t_train = t[idx,:]
u_train = u[idx,:]
v_train = v[idx,:]

In [9]:
# Noisy data
noise = 0.01        
u_train = u_train + noise*np.std(u_train)*np.random.randn(u_train.shape[0], u_train.shape[1])
v_train = v_train + noise*np.std(v_train)*np.random.randn(v_train.shape[0], v_train.shape[1])

### test to implement it

In [26]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(3,)),
    tf.keras.layers.Dense(3, activation='tanh'),
    *[tf.keras.layers.Dense(20, activation='tanh') for _ in range(8)],
    tf.keras.layers.Dense(2)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_30 (Dense)                │ (None, 3)              │            12 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 20)             │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 2)              │            42 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,074 (12.01 KB)

 Trainable params: 3,074 (12.01 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
## train() Funktion programmieren, die zuerst mit Adam Optimizer trainiert und dann mit L-BFGS-B Optimizer weiter trainiert

In [35]:
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
t_train = tf.convert_to_tensor(t_train, dtype=tf.float32)
u_train = tf.convert_to_tensor(u_train, dtype=tf.float32)
v_train = tf.convert_to_tensor(v_train, dtype=tf.float32)

In [39]:
tf.keras.optimizers.Adam(learning_rate=0.001)

In [78]:
X = tf.convert_to_tensor(tf.concat([x_train, y_train, t_train], axis=1))

In [79]:
X

<tf.Tensor: shape=(5000, 3), dtype=float32, numpy=
array([[ 7.5050507 , -0.12244898, 14.7       ],
       [ 1.4949495 , -1.3469387 ,  9.6       ],
       [ 1.5656565 , -1.0204082 , 10.2       ],
       ...,
       [ 5.4545455 , -0.53061223,  7.5       ],
       [ 1.5656565 , -0.20408164,  1.        ],
       [ 6.79798   , -0.93877554, 14.1       ]], dtype=float32)>

In [83]:
model.fit(X, 1000)

ValueError: Unrecognized data type: x=[[ 7.5050507  -0.12244898 14.7       ]
 [ 1.4949495  -1.3469387   9.6       ]
 [ 1.5656565  -1.0204082  10.2       ]
 ...
 [ 5.4545455  -0.53061223  7.5       ]
 [ 1.5656565  -0.20408164  1.        ]
 [ 6.79798    -0.93877554 14.1       ]] (of type <class 'numpy.ndarray'>)

In [27]:
# GradientTape returned None weil ich mein Model noch nicht trainiert habe, ergo sind meine Gewichte noch nicht initialisiert und ich kann deswegen auch keinen Gradienten berechnen

lambda_1 = tf.constant([0.01], dtype= tf.float32)

x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
t_train = tf.convert_to_tensor(t_train, dtype=tf.float32)

results = model(tf.concat([x_train, y_train, t_train], axis=1))

psi, p = results[:, 0:1], results[:, 1:2]

with tf.GradientTape(persistent=True) as tape:
    tape.watch([psi, p, x_train, y_train, t_train])
    u = tape.gradient(psi, y_train)
    v = tape.gradient(psi, x_train)
    p_x = tape.gradient(p, x_train)
    p_y = tape.gradient(p, y_train)
    
    u_t = tape.gradient(u, t_train)
    u_x = tape.gradient(u, x_train)
    u_y = tape.gradient(u, y_train)
    u_xx = tape.gradient(u_x, x_train)
    u_yy = tape.gradient(u_y, y_train)

    v_t = tape.gradient(-v, t_train)
    v_x = tape.gradient(-v, x_train)
    v_y = tape.gradient(-v, y_train)
    v_xx = tape.gradient(v_x, x_train)
    v_yy = tape.gradient(v_y, y_train)

f_u = u_t + lambda_1*(u*u_x + v*u_y) + p_x - lambda_1*(u_xx + u_yy)
f_v = v_t + lambda_1*(u*v_x + v*v_y) + p_y - lambda_1*(v_xx + v_yy)
    

None


In [ ]:
from typing import Union, Callable
import matplotlib.pyplot as plt

In [ ]:
from scipy.optimize import fmin_l_bfgs_b

In [ ]:
def network(loss: Union[callable, str]):
    '''funciton creates a neural network model with desired loss function'''

    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(3, activation="tanh", input_shape=(3,)),
        *[tf.keras.layers.Dense(20, activation='tanh') for _ in range(8)],
        tf.keras.layers.Dense(2)
    ])   

    model.compile(loss= loss, optimizer= "adam")

    return model

In [ ]:
def function(model, x, y, t,):
    '''Use the NN output of the stream and pressure fields to invoke the Navier Stokes equations'''

    results = model(x, y, t)
    
    psi, p = results[:, 0:1], results[:, 1:2]
        
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([psi, p, x, y, t])
        u = tape.gradient(psi, y)
        v = tape.gradient(psi, x)
        p_x = tape.gradient(p, x)
        p_y = tape.gradient(p, y)
        
        u_t = tape.gradient(u, t)
        u_x = tape.gradient(u, x)
        u_y = tape.gradient(u, y)
        u_xx = tape.gradient(u_x, x)
        u_yy = tape.gradient(u_y, y)

        v_t = tape.gradient(-v, t)
        v_x = tape.gradient(-v, x)
        v_y = tape.gradient(-v, y)
        v_xx = tape.gradient(v_x, x)
        v_yy = tape.gradient(v_y, y)

    f_u = u_t + lambda_1*(u*u_x + v*u_y) + p_x - lambda_1*(u_xx + u_yy)
    f_v = v_t + lambda_1*(u*v_x + v*v_y) + p_y - lambda_1*(v_xx + v_yy)

    return u, v, p, f_u, f_v

### Creating Model class

In [ ]:
class PINN():

    def __init__(self, x, y, t, u, v,):

        self.x = tf.Variable(x, dtype=tf.float32)
        self.y = tf.Variable(y, dtype=tf.float32)
        self.t = tf.Variable(t, dtype=tf.float32)
        self.u = tf.Variable(u, dtype=tf.float32)
        self.v = tf.Variable(v, dtype=tf.float32)
        self.null = tf.zeros_like(self.x)

        self.network()
    
        self.mse = tf.keras.losses.MeanSquaredError()

    def network(self, loss: Union[callable, str]):

        self.model = tf.keras.models.Sequential([
            tf.keras.layers.Input(shape=(3,)),
            *[tf.keras.layers.Dense(20, activation='tanh') for _ in range(8)],
            tf.keras.layers.Dense(2)
        ])   

        self.model.compile(loss= loss, optimizer= "adam")

        return self.model

    def function(self, x, y, t):
        
        x = tf.Variable(x_train, dtype=tf.float32)
        y = tf.Variable(y_train, dtype=tf.float32)
        t = tf.Variable(t_train, dtype=tf.float32)
        
        lambda_1 = tf.constant([0.01], dtype= tf.float32)

        results = self.net(tf.concat([x, y, t], axis=1))
        psi, p = results[:, 0:1], results[:, 1:2]

        with tf.GradientTape(persistent=True) as tape:
            tape.watch([psi, p])
            u = tape.gradient(psi, y)
            v = tape.gradient(psi, x)
            p_x = tape.gradient(p, x)
            p_y = tape.gradient(p, y)
            
            u_t = tape.gradient(u, t)
            u_x = tape.gradient(u, x)
            u_y = tape.gradient(u, y)
            u_xx = tape.gradient(u_x, x)
            u_yy = tape.gradient(u_y, y)

            v_t = tape.gradient(-v, t)
            v_x = tape.gradient(-v, x)
            v_y = tape.gradient(-v, y)
            v_xx = tape.gradient(v_x, x)
            v_yy = tape.gradient(v_y, y)

        f_u = u_t + lambda_1*(u*u_x + v*u_y) + p_x - lambda_1*(u_xx + u_yy)
        f_v = v_t + lambda_1*(u*v_x + v*v_y) + p_y - lambda_1*(v_xx + v_yy)

        return u, v, p, f_u, f_v
    
    def train(self, epochs):

        for epoch in range(epochs):

            with tf.GradientTape() as tape:

                u_pred, v_pred, p_pred, f_u_pred, f_v_pred = self.function(self.x, self.y, self.t)

                loss = self.mse(self.u, u_pred) + self.mse(self.v, v_pred) + self.mse(self.null, f_u_pred) + self.mse(self.null, f_v_pred)

            grads = tape.gradient(loss, self.net.trainable_variables)

            self.optimizer.apply_gradients(zip(grads, self.net.trainable_variables))

            if epoch % 100 == 0:
                print(f'Epoch {epoch}, Loss: {loss.numpy()}')

    def plot_loss(history):

        fig, ax = t.subplots()
        ax.plot(history.history['loss'], label='loss')
        ax.plot(history.history['val_loss'], label='val_loss')
        ax.legend()
        plt.show()


        

In [ ]:
model = PINN(x_train, y_train, t_train, u_train, v_train)

In [ ]:
model.train(1000)

TypeError: Argument `target` should be a list or nested structure of Tensors, Variables or CompositeTensors to be differentiated, but received None.

### PINN

In [ ]:
class PINN():

    def __init__(self, x, y, t, u, v):

        self.x = tf.convert_to_tensor(x, dtype=tf.float32)
        self.y = tf.convert_to_tensor(y, dtype=tf.float32)
        self.t = tf.convert_to_tensor(t, dtype=tf.float32)
        self.u = tf.convert_to_tensor(u, dtype=tf.float32)
        self.v = tf.convert_to_tensor(v, dtype=tf.float32)

        # Initialize NN
        self.initialize_nn()

        # Initialize parameters
        self.lambda_1 = tf.Variable([0.0], dtype= tf.float32)
        self.lambda_2 = tf.Variable([0.0], dtype= tf.float32) 


    def initialize_nn(self):
        self.network = tf.keras.Sequential([
            tf.keras.Input(shape=(3,)),
            *[tf.keras.layers.Dense(20, activation='tanh') for _ in range(8)],
            tf.keras.layers.Dense(2, activation='tanh')
        ])

    def NavierStokes(self, x, y, t):

        lambda_1 = self.lambda_1
        lambda_2 = self.lambda_2

        psi_and_p = self.network(tf.concat([x, y, t], axis=1))
        psi = psi_and_p[:, 0:1]
        p = psi_and_p[:, 1:2]

        u = tf.gradients(psi, y)[0]
        v = -tf.gradients(psi, x)[0]

        u_t = tf.gradients(u, t)[0]
        u_x = tf.gradients(u, x)[0]
        u_y = tf.gradients(u, y)[0]
        u_xx = tf.gradients(u_x, x)[0]
        u_yy = tf.gradients(u_y, y)[0]

        v_t = tf.gradients(v, t)[0]
        v_x = tf.gradients(v, x)[0]
        v_y = tf.gradients(v, y)[0]
        v_xx = tf.gradients(v_x, x)[0]
        v_yy = tf.gradients(v_y, y)[0]

        p_x = tf.gradients(p, x)[0]
        p_y = tf.gradients(p, y)[0]

        f_u = u_t + lambda_1*(u*u_x + v*u_y) + p_x - lambda_2*(u_xx + u_yy)
        f_v = v_t + lambda_1*(u*v_x + v*v_y) + p_y - lambda_2*(v_xx + v_yy)

        return u, v, p, f_u, f_v
    
    def loss_function(self):

        u_pred, v_pred, p_pred, f_u_pred, f_v_pred = self.net_NS(self.x, self.y, self.t)

        # Calculate loss of predicted velocity values
        u_loss = tf.reduce_sum(tf.square(self.u - u_pred))
        v_loss = tf.reduce_sum(tf.square(self.v - v_pred))

        # Calculate residuals of Navier-Stokes equations
        f_u_loss = tf.reduce_sum(tf.square(f_u_pred))
        f_v_loss = tf.reduce_sum(tf.square(f_v_pred))

        # Combine losses
        loss = u_loss + v_loss + f_u_loss + f_v_loss
        return loss

    def train(self, epochs):

        self.network.compile(optimizer='adam', loss=self.loss_function)
        self.network.fit([self.x, self.y, self.t], [self.u, self.v], epochs=epochs)

        # After training with adam optimzer, use L-BFGS-B optimizer to minimize the loss function 
        # -> PINN dont work solely with Adam due to Adam optimizer finding local minima
        results = tfp.optimizer.lbfgs_minimize(
            self.loss_function,
            initial_position=self.network.trainable_variables, max_iterations=1000
            )
        
        # Update model parameters after L-BFGS-B optimization
        for var, new_var in zip(self.network.trainable_variables, results.position):
            var.assign(new_var)

    def predict(self, x_star, y_star, t_star):

        x_star = tf.convert_to_tensor(x_star, dtype=tf.float32)
        y_star = tf.convert_to_tensor(y_star, dtype=tf.float32)
        t_star = tf.convert_to_tensor(t_star, dtype=tf.float32)

        u_star, v_star, p_star, _, _ = self.net_NS(x_star, y_star, t_star)

        return u_star.numpy(), v_star.numpy(), p_star.numpy()

In [ ]:
model = PINN(x_train, y_train, t_train, u_train, v_train)
model.train(epochs=1000)

Epoch 1/1000


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_90" is incompatible with the layer: expected axis -1 of input shape to have value 3, but received input with shape (None, 1)[0m

Arguments received by Sequential.call():
  • inputs=('tf.Tensor(shape=(None, 1), dtype=float32)', 'tf.Tensor(shape=(None, 1), dtype=float32)', 'tf.Tensor(shape=(None, 1), dtype=float32)')
  • training=True
  • mask=('None', 'None', 'None')